In [ ]:
import pandas as pd
import statsmodels.api as sm

In [ ]:
batting_2019 = pd.read_csv(r"D:\GitHub\Econometrics_Term_Paper\Yearly_Data\2019batting.csv")
pitching_2019 = pd.read_csv(r"D:\GitHub\Econometrics_Term_Paper\Yearly_Data\2019pitching.csv")
fielding_2019 = pd.read_csv(r"D:\GitHub\Econometrics_Term_Paper\Yearly_Data\2019fielding.csv")